## Import libraries 

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

## Function `convert_csv_2_tfrecord`: Build pipeline to read from csv file ,parse features, and write serialization of features to tfrecord file

In [3]:
# Assume to run in Graph mode!
def convert_csv_2_tfrecord(csv_file,out_file):
    def _parse_line(line):
        RECORD_DEFAULTS = [0,'A']*3+ ['A']*4+[0]*3+['A']*2
        # Decode the line into its fields
        return tf.decode_csv(line, RECORD_DEFAULTS)
        
    dataset = tf.data.TextLineDataset(csv_file).map(_parse_line)
    iterator = dataset.make_one_shot_iterator()
    features_tensor = iterator.get_next()
    writer = tf.python_io.TFRecordWriter(out_file)
    with tf.Session() as sess:
        while True:
            try:
                features = sess.run(features_tensor) 
                example = tf.train.Example(
                    features=tf.train.Features(feature=
                        {'age': _int64_feature(features[0]),
                        'workclass': _bytes_feature(features[1]),
                        'fnlwgt': _int64_feature(features[2]),
                        'education': _bytes_feature(features[3]),
                        'education-num': _int64_feature(features[4]),
                        'marital-status': _bytes_feature(features[5]),
                        'occupation': _bytes_feature(features[6]),
                        'relationship': _bytes_feature(features[7]),
                        'race': _bytes_feature(features[8]),
                        'sex': _bytes_feature(features[9]),
                        'capital-gain': _int64_feature(features[10]),
                        'capital-loss': _int64_feature(features[11]),
                        'hours-per-week': _int64_feature(features[12]),
                        'native-country': _bytes_feature(features[13]),
                        'label': _bytes_feature(features[14])
                    })
                )
                writer.write(example.SerializeToString())
            except tf.errors.OutOfRangeError:
                break
        writer.close()

## Convert CSV file to Tfrecord 

In [4]:
data = convert_csv_2_tfrecord('./adult.data','census.tfrecord')
data

In [4]:
data = convert_csv_2_tfrecord('./adult.test','census_test.tfrecord')
data

In [5]:
data